In [0]:
from google.colab import files
from urllib.request import urlretrieve
import os
import re
import pandas as pd 
import nltk
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [0]:
def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

In [0]:
download('https://storage.googleapis.com/kaggle-data-sets/551536/1124353/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1589172080&Signature=RztHLaoFQP1gM%2F0kVKPZG%2B2LySBQMBm2NS5uxLMpPPj3oufCzWQQO4RlqdKUzPNAxCA80ptp8B%2BU1WGXjnRmrTMj%2FDPMJi2fvdeJAFJKBEWIR1mW%2BMdBk49pCbastoK70X9PxKnSCkeO2EBe9npuO3FHbzVg3RYPI1Hda%2F95WbC8qq96nHR9pJsS6iNUNPiOGZgTtrD9TmMhuJIX9ByBMvPkdYP%2FKI9N4GKJukHuoxhITdoSfpxHnekCjY99%2FhkbCdH%2Bgz9v%2FECEgQOYOvTxVQrJ1bCDqdcuoZoBVP11FdhKPOiDyGHR71PRx9FiC3ca4r%2FBkqi0owwJgkqd7mpq1w%3D%3D&response-content-disposition=attachment%3B+filename%3Dcoronavirus-covid19-tweets.zip','COVID19_Tweets.zip')

Download file... COVID19_Tweets.zip ...
File downloaded


In [0]:
!unzip COVID19_Tweets.zip

Archive:  COVID19_Tweets.zip
  inflating: 2020-03-00 Coronavirus Tweets (pre 2020-03-12).CSV  
  inflating: 2020-03-12 Coronavirus Tweets.CSV  
  inflating: 2020-03-13 Coronavirus Tweets.CSV  
  inflating: 2020-03-14 Coronavirus Tweets.CSV  
  inflating: 2020-03-15 Coronavirus Tweets.CSV  
  inflating: 2020-03-16 Coronavirus Tweets.CSV  
  inflating: 2020-03-17 Coronavirus Tweets.CSV  
  inflating: 2020-03-18 Coronavirus Tweets.CSV  
  inflating: 2020-03-19 Coronavirus Tweets.CSV  
  inflating: 2020-03-20 Coronavirus Tweets.CSV  
  inflating: 2020-03-21 Coronavirus Tweets.CSV  
  inflating: 2020-03-22 Coronavirus Tweets.CSV  
  inflating: 2020-03-23 Coronavirus Tweets.CSV  
  inflating: 2020-03-24 Coronavirus Tweets.CSV  
  inflating: 2020-03-25 Coronavirus Tweets.CSV  
  inflating: 2020-03-26 Coronavirus Tweets.CSV  
  inflating: 2020-03-27 Coronavirus Tweets.CSV  
  inflating: 2020-03-28 Coronavirus Tweets.CSV  
  inflating: Countries.CSV           
  inflating: Hashtags.CSV         

In [0]:
df=pd.read_csv('2020-03-28 Coronavirus Tweets.CSV')
df=df[df['lang']=='en']
df=df[['text']]

In [0]:
nltk.download('stopwords')

tweets = [] 
tweets_wrdlist = []

for i in df['text'][200000:300000]:
  tweet = re.sub('[^a-zA-Z]', ' ', i)
  tweet = tweet.lower()
  tweet = tweet.split()
  prtstm = PorterStemmer()
  tweet = [prtstm.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
  tweets_wrdlist.append(tweet)
  tweet = ' '.join(tweet)
  tweets.append(tweet)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
files.upload()

Saving negative_words.txt to negative_words.txt
Saving positive_words.txt to positive_words.txt


{'negative_words.txt': b'2-faced\r\n2-faces\r\nabnormal\r\nabolish\r\nabominable\r\nabominably\r\nabominate\r\nabomination\r\nabort\r\naborted\r\naborts\r\nabrade\r\nabrasive\r\nabrupt\r\nabruptly\r\nabscond\r\nabsence\r\nabsent-minded\r\nabsentee\r\nabsurd\r\nabsurdity\r\nabsurdly\r\nabsurdness\r\nabuse\r\nabused\r\nabuses\r\nabusive\r\nabysmal\r\nabysmally\r\nabyss\r\naccidental\r\naccost\r\naccursed\r\naccusation\r\naccusations\r\naccuse\r\naccuses\r\naccusing\r\naccusingly\r\nacerbate\r\nacerbic\r\nacerbically\r\nache\r\nached\r\naches\r\nachey\r\naching\r\nacrid\r\nacridly\r\nacridness\r\nacrimonious\r\nacrimoniously\r\nacrimony\r\nadamant\r\nadamantly\r\naddict\r\naddicted\r\naddicting\r\naddicts\r\nadmonish\r\nadmonisher\r\nadmonishingly\r\nadmonishment\r\nadmonition\r\nadulterate\r\nadulterated\r\nadulteration\r\nadulterier\r\nadversarial\r\nadversary\r\nadverse\r\nadversity\r\nafflict\r\naffliction\r\nafflictive\r\naffront\r\nafraid\r\naggravate\r\naggravating\r\naggravation\r

In [0]:
fileP=open('positive_words.txt','r')
pos_words=fileP.readlines()
pos_words=set([prtstm.stem(word[0:-1]) for word in pos_words if not word in set(stopwords.words('english'))])

In [0]:
fileN=open('negative_words.txt','r')
neg_words=fileN.readlines()
neg_words=set([prtstm.stem(word[0:-1]) for word in neg_words if not word in set(stopwords.words('english'))])

In [0]:
labels=[]

for tweet in tweets_wrdlist:
  pos=0
  neg=0
  for word in tweet:
    if word in pos_words:
      pos+=1
    if word in neg_words:
      neg+=1
  if pos>neg:
    labels.append(1)
  else:
    labels.append(0)

In [0]:
countvec = CountVectorizer(max_features = 1000)
features = countvec.fit_transform(tweets).toarray()

In [0]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.3)
model = RandomForestClassifier(n_estimators=33)
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=33,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred=model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8174666666666667

In [0]:
#tweet_str='This government sucks, how can they lock millions of people in their homes against their will.'
#tweet_str='Very bad decision by the government'
tweet_str='COVID19 lockdown might be something disheartening for humans but for the earth and nature health, it is turning out great'

tweet_str = re.sub('[^a-zA-Z]', ' ', tweet_str) 
tweet_str = tweet_str.lower() 
tweet_str = tweet_str.split()
prtstm = PorterStemmer() 
tweet_str = [prtstm.stem(word) for word in tweet_str if not word in set(stopwords.words('english'))]
tweet_str = ' '.join(tweet_str)
tweet_str=[tweet_str]
feat = countvec.transform(tweet_str).toarray()

ans=model.predict(feat)

if ans[0]==1:
  print('Positive Tweet')
else:
  print('Negative Tweet')

Positive Tweet
